In [31]:
import math
from Phase_1.src.Utils.SimplePositionalIndex import SimplePositionalIndex
from Phase_1.src.Utils.StopWord import Document
from Phase_1.src.Utils.utilities import read_file, print_results
from Phase_2.src.config import Config

class DevelopedPositionalIndex(SimplePositionalIndex):
    def __init__(self, configurations):
        super().__init__(configurations)
        self.total_number_of_documents = len(self.Documents)
        self.document_term_tfidf_dictionary = {}
        self.build_updated_positional_index()
        if configurations.get_config('champions_list'):
            self.champions_list = self.build_champions_list()

    def build_updated_positional_index(self):
        for WORD in self.positional_index_structure.keys():
            for DOC_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                DICTIONARY['tf idf'] = self.get_tf_value(WORD, DOC_URL) * self.get_idf_value(WORD)
                if DOC_URL not in self.document_term_tfidf_dictionary.keys():
                    self.document_term_tfidf_dictionary[DOC_URL] = {WORD: DICTIONARY['tf idf']}
                else:
                    self.document_term_tfidf_dictionary[DOC_URL][WORD] = DICTIONARY['tf idf']

    def get_tf_value(self, word, url):
        return math.log(1 + self.positional_index_structure[word]['indexes'][url]['number of occurrences in document'])

    def get_idf_value(self, word):
        return math.log(self.total_number_of_documents / len(self.positional_index_structure[word]['indexes']))

    def build_champions_list(self):
        champions_list = {}
        for WORD in self.positional_index_structure.keys():
            url_tf_dictionary = {}
            for DOC_URL, DICTIONARY in self.positional_index_structure[WORD]['indexes'].items():
                url_tf_dictionary[DOC_URL] = self.get_tf_value(WORD, DOC_URL)
            champions_list_size = int(
                self.config.get_config('champions_lists_ratio') * len(self.positional_index_structure[WORD]['indexes']))
            champions_list[WORD] = sorted(url_tf_dictionary, key=lambda item: item[1], reverse=True)[
                                   :champions_list_size]
        return champions_list


config = Config()
config.set_config('documents_path', '/Volumes/Farshid_SSD/Projects/University/information retrieval/Phase_1/assets/IR_data_news_12k.json')
docs_url, docs_title, docs_content = read_file(config.get_config('documents_path'))

config.set_config('documents',
                  [Document(content, url, title) for url, title, content in zip(docs_url, docs_title, docs_content)])
config.set_config('champions_list', True)

pos_index = DevelopedPositionalIndex(config)


In [32]:
import pickle

def save_index(configurations):
    with open(f'../{configurations.get_config("dump_path")}/updated_pos_index.pkl', 'w') as file_to_write:
        pickle.dump(pos_index, file_to_write, pickle.HIGHEST_PROTOCOL)
        print(f'saved index with name updated_pos_index.pkl')

def load_index(configurations):
    index = None
    with open(f'../{configurations.get_config("dump_path")}/updated_pos_index.pkl', 'rb') as file_to_read:
        index = pickle.load(file_to_read)
    return index

# save_index(config)
# pos_index = load_index(config)

In [33]:
from collections import Counter
from math import log

from Phase_1.src.Utils.utilities import preprocess_pipeline


class QueryHandler:
    def __init__(self, positional_index, config):
        self.positional_index = positional_index
        self.config = config

    def answer_query(self, query):
        query = preprocess_pipeline(query)
        print(f'Query: {query}\n\n\n')
        terms = query.split()
        vector_values = self.tf_idf_calculate(terms, self.config.get_config('champions_list'))
        scores = self.calculate_scores(vector_values)
        results_url = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)[
                           :self.config.get_config('documents_to_show')])
        total_results = self.generate_total_results(results_url)
        return total_results

    def generate_total_results(self, results_url):
        total_results = []
        for URL, SCORE in results_url.items():
            result = self.positional_index.url_to_information[URL]
            result['score'] = SCORE
            result['url'] = URL
            total_results.append(result)
        return total_results

    def tf_idf_calculate(self, terms, champions_list):
        vector_values = {}
        tf_values = Counter(terms)
        for term in terms:
            for DOC_URL in self.get_search_dict_list(champions_list, term):
                if DOC_URL not in vector_values.keys():
                    vector_values[DOC_URL] = {}
                vector_values[DOC_URL][term] = \
                    (log(1 + tf_values[term])) * self.positional_index.get_idf_value(term)
        return vector_values

    def get_search_dict_list(self, champions_list, term):
        if champions_list and term in self.positional_index.champions_list.keys():
            return self.positional_index.champions_list[term]
        if term in self.positional_index.positional_index_structure.keys():
            return self.positional_index.positional_index_structure[term]['indexes']
        return None

    @staticmethod
    def cosine_similarity(v1, v2):
        dot_product = 0
        for term in v1.keys():
            if term in v2.keys():
                dot_product += v1[term] * v2[term]
        magnitude_v1 = 0
        for term in v1.keys():
            magnitude_v1 += v1[term] ** 2
        magnitude_v2 = 0
        for term in v2.keys():
            magnitude_v2 += v2[term] ** 2
        return dot_product / (magnitude_v1 ** 0.5) / (magnitude_v2 ** 0.5)

    def calculate_scores(self, vector_values):
        scores = {}
        for DOC_URL, TERM_SCORES in vector_values.items():
            scores[DOC_URL] = self.cosine_similarity(vector_values[DOC_URL],
                                                     self.positional_index.document_term_tfidf_dictionary[DOC_URL])
        return scores



In [34]:
query_handler = QueryHandler(pos_index, config)
print_results(query_handler.answer_query('مجلس'))

Query: مجلس



score: 0.1958694665272313,
url: https://www.farsnews.ir/news/14001208001040/تست-کرونای-رئیس-مجلس-مثبت-اعلام-شد,
title: تست کرونای رئیس مجلس مثبت اعلام شد
content: 
به گزارش خبرنگار خبرگزاری فارس، تست کرونای محمد باقر قالیباف رئیس مجلس شورای اسلامی مثبت اعلام شد. بنا بر اعلام نزدیکان رئیس مجلس، حال وی مساعد بوده و در دفتر کار مجلس، در قرنطینه و در حال پیگیری امور است. انتهای پیام/



*********************************************************


score: 0.17313213293781657,
url: https://www.farsnews.ir/news/14001130000387/مجلس-فردا-با-محوریت-بودجه-سال-آینده-نشست-غیرعلنی-دارد,
title: مجلس فردا با محوریت بودجه سال آینده نشست غیرعلنی دارد
content: 
به گزارش خبرنگار خبرگزاری فارس، مجلس شورای اسلامی فردا یکشنبه (1 اسفند) وارد بررسی جزییات لایحه بودجه سال ۱۴۰۱ کل کشور می‌شود. احتمالا صحن علنی مجلس قبل از نشست علنی یک نشست غیر علنی با حضور مسئولان اقتصادی و بودجه‌ای دولت داشته باشد. به گزارش فارس، کمیسیون تلفیق بودجه هفته گذشته لایحه بودجه 1401 را به هیئت رئیسه مجلس ارسال کرده بود ت

In [35]:
print_results(query_handler.answer_query('تیم استقلال'))

Query: ت استقلال



score: 0.2849273035104969,
url: https://www.farsnews.ir/news/14001128000707/تساوی-پرگل-استقلال-و-پرسپولیس-در-رده-جوانان,
title: تساوی پرگل استقلال و پرسپولیس در رده جوانان
content: 
به گزارش خبرگزاری فارس و به نقل از باشگاه استقلال، تیم فوتبال رده جوانان استقلال امروز (پنجشنبه) در هفته بیست و چهارم لیگ پایه میزبان تیم پرسپولیس بود که این بازی با تساوی دو بر دو به پایان رسید. برای استقلال رضا بهرامیان و محمدرضا کشاورز در این بازی گلزنی کردند. جوانان استقلال با این تساوی ۴۱ امتیازی شدند و در رده پنجم جدول رده‌بندی قرار گرفتند. انتهای پیام/



*********************************************************


score: 0.28351070254397426,
url: https://www.farsnews.ir/news/14001027000699/باشگاه-استقلال-ابتلای-مجیدی-به-کرونا-صحت-ندارد,
title: باشگاه استقلال: ابتلای مجیدی به کرونا صحت ندارد
content: 
به گزارش خبرگزاری فارس، با اعلام باشگاه استقلال، تست کرونای فرهاد مجیدی منفی است و خبر ابتلای سرمربی استقلال به ویروس کرونا صحت ندارد. این در حالی است که سرمربی استقلال امروز در نشست 

In [36]:
print_results(query_handler.answer_query('یوفا'))

Query: یوفا



score: 0.3362926389828227,
url: https://www.farsnews.ir/news/14001217000240/شکایت-روسیه-از-یوفا-و-فیفا-به-CAS,
title: شکایت روسیه از یوفا و فیفا به CAS
content: 
به گزارش خبرگزاری فارس، سایت تاس روسیه اعلام کرد که سرانجام پس از گذشت چند روز، فدراسیون فوتبال روسیه شکایت خود را علیه فیفا و یوفا نهایی کرد. در تاریخ 9 اسفند،‌ فیفا و یوفا به طور رسمی تیم‌ ملی و تیم‌های باشگاهی روسیه را از حضور در رقابت‌های مختلف محروم کردند. در تاریخ 3 مارس (12 اسفند) فدراسیون فوتبال روسیه اعلام داشت که به این رای شکایت خواهد کرد. این شکایت البته چند روزی طول کشید تا اینکه شب گذشته به صورت رسمی این اقدام نهایی شد. خبرگزاری تاس روسیه نوشت: یک فرد رسمی در فدارسیون فوتبال روسیه از شکایت آنها علیه فیفا و یوفا به CAS (دادگاه حکمیت ورزش) خبر داد. با توجه به این محرومیت روسیه از حضور در پلی‌آف جام جهانی 2022 قطر محروم خواهد شد و باید دید چه اتفاقاتی تا تاریخ 4  فروردین رخ خواهد داد. پیش از این قرار بود که تیم ملی روسیه با تیم ملی لهستان دیدار کند و برنده این بازی با برنده جدال چک و سوئد روبه‌رو شود.

In [37]:
print_results(query_handler.answer_query('تیبو کورتوا'))

Query: تیبو کورتوا



score: 0.5619002853324593,
url: https://www.farsnews.ir/news/14001213000563/ستاره-بلژیکی-بهترین-بازیکن-ماه-رئال-مادرید,
title: ستاره بلژیکی بهترین بازیکن ماه رئال مادرید
content: 
به گزارش خبرگزاری فارس، تیبو کورتوا سنگربان بلژیکی باشگاه رئال مادرید به عنوان بهترین ماه فوریه باشگاه رئال مادرید انتخاب شد. وی همچنین به عنوان یکی از نامزدهای کسب عنوان بهترین بازیکن ماه لالیگا هم انتخاب شده است. انتهای پیام/



*********************************************************


score: 0.21836609121543407,
url: https://www.farsnews.ir/news/14001218000985/ترکیب-منتخب-رئال-و-پاریس-به-بهانه-دیدار-حساس-امشب-در-لیگ-قهرمانان-عکس,
title: ترکیب منتخب رئال و پاریس به بهانه دیدار حساس امشب در لیگ قهرمانان+عکس
content: 
به گزارش خبرگزاری فارس، امشب از ساعت 23:30 به وقت ایران، رئال مادرید در ورزشگاه سانتیاگو برنابئو میزبان پاری سن ژرمن است و این بازی ساعت 23:30 به وقت ایران آغاز می شود. دیدار رفت با گل دیرهنگام کیلیان ام باپه مهاجم پاری سن ژرمن با یک گل به سود این تیم به پایان رسیده است و

In [38]:
print_results(query_handler.answer_query('تحریم جهانی علیه کشور جهان سومی ایران در جمعه'))

Query: تحر جهان کشور جه سوم ایر جمعه



score: 0.33007006285803114,
url: https://www.farsnews.ir/news/14001107000679/واکنش-رسن-بعد-از-باخت-به-ایران-هافبک-سابق-پرسپولیس-ناامید-نیست,
title: واکنش رسن بعد از باخت به ایران/هافبک سابق پرسپولیس ناامید نیست
content: 
به گزارش خبرگزاری فارس، تیم ملی کشورمان امشب در دور هفتم مسابقات انتخابی جام جهانی 2022 مقابل عراق قرار گرفت و با نتیجه یک بر صفر به پیروزی رسید تا صعودش به جام جهانی قطر قطعی شود. بشار رسن هافبک سابق پرسپولیس بعد از این باخت، گفت: ما در این بازی عملکرد خوبی داشتیم. با وجود باخت ما همچنان امیدوار هستیم. وی افزود: ما هنوز 3 بازی پیش رو داریم و می توانیم این باخت را جبران کنیم و سهمیه صعود به جام جهانی به عنوان تیمی سومی گروه را کسب کنیم. انتهای پیام/  



*********************************************************


score: 0.3147778303974535,
url: https://www.farsnews.ir/news/14001204000081/با-حکم-AFC-تحریم-میزبانی-همگروه-ایران-برداشته-شد,
title: با حکم AFC تحریم میزبانی همگروه ایران برداشته شد
content: 
به گزارش خبرگزاری فارس، تیم 